## Step 1: Load the data

Load data from the csv and the images

In [27]:
#AWS

import pickle
import os
from urllib.request import urlretrieve
from zipfile import ZipFile

def download(url, file):
    """
    Download file from <url>
    :param url: URL to file
    :param file: Local file path
    """
    if not os.path.isfile(file):
        print('Downloading ' + file + '...')
        urlretrieve(url, file)
        print('Download Finished')
        #Unzip the downloaded file to get pickled data
        zip = ZipFile('data.zip')
        zip.extractall()

# Downloading the training and test dataset.
download('https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip', 'data.zip')

# Wait until you see that all files have been downloaded.
print('All files downloaded.')



All files downloaded.


In [28]:
import csv
import cv2
import numpy as np

lines =[]
#with open('../Behavioural Cloning Data/driving_log.csv') as csvfile:
with open('data/driving_log.csv') as csvfile:
#with open('../data/driving_log_2.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
images=[]
measurements=[]
count=0
for line in lines[1:]:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = 'data/IMG/' + filename
    #current_path = source_path
    image = cv2.imread(current_path)
    #print(image.shape)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    if (abs(measurement)>0.1):
        count=count+1
        image_flipped = np.fliplr(image)
        measurement_flipped = -measurement
        images.append(image_flipped)
        measurements.append(measurement_flipped)
        images.append(image_flipped)
        measurements.append(measurement_flipped)
    if (abs(measurement)>0.3):
        count=count+1
        image_flipped = np.fliplr(image)
        measurement_flipped = -measurement
        images.append(image_flipped)
        measurements.append(measurement_flipped)
        images.append(image)
        measurements.append(measurement)
        
    
X_train = np.array(images)
y_train = np.array(measurements)
'''X_train = np.array(X_train)
y_train = np.array(y_train)'''
print('Hello')
print (X_train.shape)
print(count)
print(len(lines))
print(len(images))

Hello
(13058, 160, 320, 3)
2511
8037
13058


## Step 2: Design and Test a Model Architecture

Model architecture

In [31]:
%config IPCompleter.greedy=True
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Activation
from keras.layers import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
#normalization
model.add(Lambda(lambda x:x/255.0-0.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((75,25),(0,0))))


#Layer 1: Convolution
model.add(Convolution2D(6, 5, 5,border_mode='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Activation('elu'))


#Layer 2: Convolution
model.add(Convolution2D(16, 5, 5,border_mode='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Activation('elu'))

#Layer 3: Convolution
model.add(Convolution2D(18, 5, 5,border_mode='valid'))
model.add(MaxPooling2D((1, 2)))
model.add(Activation('elu'))

#Layer 4: Convolution
model.add(Convolution2D(20, 3, 3,border_mode='valid'))
model.add(MaxPooling2D((1, 2)))
model.add(Activation('relu'))

#Layer 5: Convolution
model.add(Convolution2D(8, 3, 3,border_mode='valid'))
#model.add(MaxPooling2D((2, 2)))
model.add(Activation('elu'))



model.add(Flatten())
model.add(Dense(400))
model.add(Activation('elu'))
model.add(Dense(200))
model.add(Activation('elu'))
model.add(Dense(80))
model.add(Activation('elu'))
model.add(Dense(1))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_11 (Lambda)               (None, 160, 320, 3)   0           lambda_input_11[0][0]            
____________________________________________________________________________________________________
cropping2d_11 (Cropping2D)       (None, 60, 320, 3)    0           lambda_11[0][0]                  
____________________________________________________________________________________________________
convolution2d_51 (Convolution2D) (None, 56, 316, 6)    456         cropping2d_11[0][0]              
____________________________________________________________________________________________________
maxpooling2d_41 (MaxPooling2D)   (None, 28, 158, 6)    0           convolution2d_51[0][0]           
___________________________________________________________________________________________

## Training the model

In [32]:
model.compile(loss='mse', optimizer='adam')
model.fit(X_train,y_train, validation_split=0.2, shuffle=True, nb_epoch=1)

model.save('model.h5')

print('Done')

Train on 10446 samples, validate on 2612 samples
Epoch 1/1
10446/10446 [==============================] - 23s - loss: 0.0236 - val_loss: 0.0199
Done


## Step 3: Training





## Step 4: Validation
